# Wrangle Data

In [1]:
import pandas as pd
import datetime
import sqlite3
import zipfile

### Import Data

Crime Data:
- ZIP File.

In [2]:
crime_zip = zipfile.ZipFile('../data/dc-crime-data/dc-crime-data.csv.zip', mode='r')
crime_csv = crime_zip.open('dc-crime-data.csv')
crime_df = pd.read_csv(crime_csv)

Weather Data
- DB Name: weather_data
- Table Name: weather_data

In [3]:
conn = sqlite3.connect('../data/weather-data/weather_data.db')
weather_df = pd.read_sql('''select * from weather_data''', conn)
conn.close()

Census Data

- DB Name: census_bg
- Table Name: census_blockgroup

In [4]:
conn = sqlite3.connect('../data/census-data/census_bg.db')
census_df = pd.read_sql('''select * from census_blockgroup''', conn)
conn.close()

### Wrangle Data

Crime Data:
- Create year, month, day, and time of day variables. TOD staddles mid-night, means interpretation of TOD is not continuous for the Midnight category.

In [5]:
def assign_tod(row):
    '''
    Assign time of day (TOD) based on the hour.
    '''
    try:
        timestamp = pd.Timestamp(row['START_DATE'])

        year = timestamp.year
        month = timestamp.month
        day = timestamp.day
        start_hour = timestamp.time().hour

        if 23 <= start_hour:
            time_of_day = 'Midnight'
        if 0 <= start_hour < 2:
            time_of_day = 'Midnight'
        elif 2 <= start_hour < 5:
            time_of_day = 'Late Night'
        elif 5 <= start_hour < 8:
            time_of_day = 'Early Morning'
        elif 8 <= start_hour < 11:
            time_of_day = 'Morning'
        elif 11 <= start_hour < 14:
            time_of_day = 'Afternoon'
        elif 14 <= start_hour < 17:
            time_of_day = 'Mid Afternoon'
        elif 17 <= start_hour < 20:
            time_of_day = 'Evening'
        elif 20 <= start_hour < 23:
            time_of_day = 'Night'
    except:
        year, month, day, time_of_day = '','','',''

    return year, month, day, time_of_day

In [6]:
crime_df[['year','month','day','tod']] = crime_df.apply(assign_tod, axis=1, result_type='expand')

Weather Data:
- Remove negative time values, that is, time before 1970.
- Convert date-time to a format to match to the crime data, named "crime_time".
- Rename variables.
- Delete those that are non-numeric (one could encoding these variables, theoretically).

In [7]:
def convertTime(row):
    '''
    Convert time elapsed from 1/1/1970 to YYYY-MM-DTHH:MM:SS.MMM format.
    '''
    time = datetime.datetime.fromtimestamp(row).strftime('%Y-%m-%dT%H:%M:%S.000')

    return time

In [8]:
#There are negative time values that must be removed.
weather_df = weather_df[weather_df['currently_time'] > 0]

#Convert time to format shared by crime data.
weather_df['crime_time_format'] = weather_df['currently_time'].apply(convertTime)

#Rename and drop columns.
rename_cols = dict(currently_apparentTemperature = 'apparent_temp',
                   currently_cloudCover = 'cloud_cover',
                   currently_dewPoint = 'dew_point',
                   currently_humidity = 'humidity',
                   currently_icon = 'icon',
                   currently_precipIntensity = 'percip_intensity',
                   currently_precipProbability = 'percip_probability',
                   currently_precipType = 'percip_type',
                   currently_pressure = 'pressure',
                   currently_summary = 'summary',
                   currently_temperature = 'temperature',
                   currently_time = 'time',
                   currently_uvIndex = 'uv_index',
                   currently_visibility = 'visibility',
                   currently_windBearing = 'wind_bearing',
                   currently_windGust = 'wind_gust',
                   currently_windSpeed = 'wind_speed',
                   latitude = 'weather_latitude',
                   longitude = 'weather_longitude')
drop_cols = ['index','code','summary','icon','error','percip_type']

weather_df.rename(columns=rename_cols, inplace=True)
weather_df.drop(labels=drop_cols, axis='columns', inplace=True)

Census Data

In [9]:
#Fill NA values and values less than 0 with the mean of values greater than zero.
columns = ['TotalPop','TPopMargin','UnWgtSampleCtPop','PerCapitaIncome','MedianHouseholdInc',
           'MedianAge','HousingUnits','UnweightedSampleHousingUnits']

for col in columns:
    #Edit string (object) columns to numeric (float).
    if census_df[col].dtypes == 'object':
        numeric_column = pd.to_numeric(census_df[col], errors = 'coerce')
        census_df[col] =  numeric_column

    #Calculate Mean.
    mean = census_df[census_df[col] > 0][col].mean()

    #Fill NA with a dictionary of column name and the mean value.
    census_df.fillna(value={col: mean}, inplace=True)

    #Replace values less than zero with the mean.
    census_df[census_df[col] < 0] = mean

#Keep population values greater than zero.
census_df = census_df.loc[census_df['TotalPop'] > 0]

#Reformat columns and rename year column.
census_df['BlockGroup'] = census_df['BlockGroup'].astype(str).replace(']]', '', regex=True)
census_df['BlockGroup'] = census_df['BlockGroup'].astype(str).replace('\.0', '', regex=True)
census_df['Tract'] = census_df['Tract'].astype(str).replace('\.0', '', regex=True)
census_df['Tract'] = census_df['Tract'].apply(lambda x: x.zfill(6))
census_df['Year'] = census_df['Year'].astype(str).replace('\.0', '', regex=True)
census_df.rename(columns=dict(Year = 'census_year'), inplace=True)

#Create an index to merge with crime data.
census_df['index'] = census_df['Tract'] + census_df['BlockGroup'] + census_df['census_year']
census_df_nodup = census_df.drop_duplicates(subset='index')

### Merge Weather and Crime Data

- Merge weather and crime data before aggregating.

In [10]:
crime_weather_mr = crime_df_wr.merge(weather_df_wr,
                                     how='left',
                                     left_on=['LATITUDE','LONGITUDE','START_DATE'],
                                     right_on=['weather_latitude','weather_longitude','crime_time_format'])

NameError: name 'crime_df_wr' is not defined

### Aggregate Crime/Weather Data

- Set "by_crime_type" to indicate whether to aggregate all crimes to by crime table (violent/non-violent).
- Count REPORT_DAT, since data is collected from MPD based on report date. This is like the most reliable variable to count instances of reported crime.
- For all variables except REPORT_DAT, we want to average. In this case, we average weather data across the TOD blocks.

In [ ]:
by_crime_type = False

In [ ]:
#Identify variables for aggregation. Count instances of REPORT_DAT and average weather variables.
agg_vars = ['REPORT_DAT', 'apparent_temp', 'cloud_cover', 'dew_point', 'humidity',
            'percip_intensity', 'percip_probability', 'pressure', 'temperature', 'uv_index',
            'visibility', 'wind_bearing', 'wind_gust', 'wind_speed']

agg_dict = dict()

#Set aggregation method for each variable.
for var in agg_vars:
    if var == 'REPORT_DAT':
        agg_dict[var] = 'size'
    else:
        agg_dict[var] = 'mean'

#Based on user choice, aggregate by offensegroup or by all offenses.
if by_crime_type == False:
    crime_weather_agg = crime_weather_df.groupby(by=['BLOCK_GROUP','year','month','day','tod'], as_index=False).agg(agg_dict)
else:
    crime_weather_agg = crime_weather_df.groupby(by=['BLOCK_GROUP','year','month','day','tod','offensegroup'], as_index=False).agg(agg_dict)

#Rename REPORT_DAT to crime_counts.
rename_cols = dict(REPORT_DAT = 'crime_counts')

crime_weather_agg.rename(columns=rename_cols, inplace=True)

#Create Index variable to merge with US Census data.
crime_weather_agg['index'] = crime_weather_agg['BLOCK_GROUP'] + crime_weather_agg['year'].astype('str')
crime_weather_agg['index'] = crime_weather_agg['index'].str.replace(" ","")

### Merge Weather-Crime and Census Data

In [ ]:
if by_crime_type == False:
    crime_weather_census = crime_weather_agg.merge(census_df_wr, how='left', on='index')
else:
    crime_weather_census = crime_weather_agg.merge(census_df_wr, how='left', on='index')

### Calculate Crime Rates

- Calculate crime rates per 100,000 people.
- Remove instances of blank Census population data.

$$\text{Crime Rate}_{year, month, day, tod, bg, crime} = \frac{\text{Total Crime Count}_{year,month,day,tod,bg,crime}}{\text{Total Population}_{year,month,day,tod,bg,crime}} * 100,000$$

In [ ]:
#Remove blank Census data
crime_weather_census_na = crime_weather_census.dropna(axis='index', how='any', subset=['TotalPop']).reset_index(drop=True)

#Calculate Crime Rates per 100,000 people
crime_weather_census_na['crime_rate'] = (crime_weather_census['crime_counts'] / crime_weather_census['TotalPop'])*100000